In [1]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai langchain-qdrant fastembed langchain-community qdrant-client langgraph

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.9 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.0 MB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 1.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.39.0 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_2 = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=env.GOOGLE_API_KEY)

In [3]:
from qdrant_client.http.models import Distance

collection_name = "mcu_packages"
dimension = 3072
distance = Distance.COSINE

# Create Vector Data

In [4]:
# load mcu.json data
import json

with open("mcu.json", "r") as f:
    mcu_data = json.load(f)

print(mcu_data[0])

{'id': '50924', 'name': 'Siloam Silver Package', 'description': '<p>Basic examinations that must be routinely checked include complete blood count, liver function, kidney function, uric acid, blood sugar, heart, lungs, plus abdominal ultrasound.</p>\n<p style="line-height: 1.2;" data-sourcepos="3:1-3:52">&nbsp;</p>\n<p style="line-height: 1.2;" data-sourcepos="3:1-3:52"><strong>To order multiple packages of the same product:</strong></p>\n<ol>\n<li style="line-height: 1.2;" data-sourcepos="3:1-3:52">Add Item to cart</li>\n<li style="line-height: 1.2;" data-sourcepos="3:1-3:52">Click \'Set Up Reservation\' button, view the selected products, and click your name.</li>\n<li style="line-height: 1.2;" data-sourcepos="6:1-6:57">On the \'Choose Patient\' page, select the name of another patient who has been registered, or click \'Add New Patient\'.</li>\n<li style="line-height: 1.2;" data-sourcepos="7:1-7:73">You can add any number of patients, as needed.</li>\n</ol>', 'price': 1900000}


In [ ]:
# Optionally, you can use FastEmbed for embeddings
# from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
# embeddings = FastEmbedEmbeddings(cache_dir="./embedding_cache", model_name="jinaai/jina-embeddings-v2-base-en")
# # https://qdrant.github.io/fastembed/examples/Supported_Models/#supported-text-embedding-models

In [5]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [6]:
from qdrant_client.http.models import VectorParams

if(client.collection_exists(collection_name=collection_name) == False):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=distance),
    )

In [ ]:
from qdrant_client.models import PointStruct
import uuid
i = 0
for row in mcu_data:
    i += 1
    text = f"Package Name: {row['name']}, Description: {row['description']}"
    emb = embeddings_2.embed_query(text)
    print(i)
    client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=str(uuid.uuid4()),  # Generate a unique ID for each point
                vector=emb, 
                payload={
                    "page_content": text,
                    "metadata": {
                            "id": row['id'],
                            "name": row['name'],
                            "description": row['description'],
                    },
                },
            )
        ],
    )
    print(text)

1
Package Name: Siloam Silver Package, Description: <p>Basic examinations that must be routinely checked include complete blood count, liver function, kidney function, uric acid, blood sugar, heart, lungs, plus abdominal ultrasound.</p>
<p style="line-height: 1.2;" data-sourcepos="3:1-3:52">&nbsp;</p>
<p style="line-height: 1.2;" data-sourcepos="3:1-3:52"><strong>To order multiple packages of the same product:</strong></p>
<ol>
<li style="line-height: 1.2;" data-sourcepos="3:1-3:52">Add Item to cart</li>
<li style="line-height: 1.2;" data-sourcepos="3:1-3:52">Click 'Set Up Reservation' button, view the selected products, and click your name.</li>
<li style="line-height: 1.2;" data-sourcepos="6:1-6:57">On the 'Choose Patient' page, select the name of another patient who has been registered, or click 'Add New Patient'.</li>
<li style="line-height: 1.2;" data-sourcepos="7:1-7:73">You can add any number of patients, as needed.</li>
</ol>
2
Package Name: Stroke Basic Screening Package, Desc

# Create Tool

In [8]:
from langchain_qdrant import QdrantVectorStore
def get_retriever():

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings_2,
    )
    
    return vector_store.as_retriever()

In [9]:
from langchain_core.tools import tool
from typing import Annotated, List

@tool
def search_mcu_packages(query: Annotated[str, "search query must contain keywords related to MCU packages"]) -> List[str]:
    """Search for MCU packages by name or description."""
    retriever = get_retriever()
    results = retriever.invoke(query, k=10)
    return [result.page_content for result in results]

In [10]:
search_mcu_packages("gula darah, diabetes, paket medical check up, siloam")

C:\Users\ACER\AppData\Local\Temp\ipykernel_15644\1917418210.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_mcu_packages("gula darah, diabetes, paket medical check up, siloam")


['Package Name: Siloam Ruby Package, Description: <p>Basic examination to check routinely, including full blood count, liver function, kidney function, uric acid, cholesterol profile, fasting glucose, heart, and lungs examination.</p>\n<p data-sourcepos="3:1-3:52"><strong>To order multiple packages of the same product:</strong></p>\n<ol>\n<li data-sourcepos="3:1-3:52">Add Item to cart</li>\n<li data-sourcepos="3:1-3:52">Click \'Set Up Reservation\' button, view the selected products, and click your name.</li>\n<li data-sourcepos="6:1-6:57">On the \'Choose Patient\' page, select the name of another patient who has been registered, or click \'Add New Patient\'.</li>\n<li data-sourcepos="7:1-7:73">You can add any number of patients, as needed.</li>\n</ol>\n<p>The price of your MCU already includes consultation services for the results through the <em>24-Hour Doctor Chat</em>&nbsp;on the MySiloam app&nbsp;or through the official Siloam Teleconsultation WhatsApp number +62 852 1601 8181&nbs

# Create Agent

In [11]:
# access the Google Gemini API
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=env.GOOGLE_API_KEY,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides information about Siloam hospitals."),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [12]:
chain.invoke({"question": "Saya mau cek gula darah di siloam, ada paket apa aja ya ?"})

AIMessage(content='Halo! Untuk pengecekan gula darah di Siloam Hospitals, terdapat beberapa pilihan paket yang tersedia, tergantung pada kebutuhan dan jenis pemeriksaan yang Anda inginkan. Berikut beberapa opsi yang mungkin tersedia:\n\n1.  **Paket Pemeriksaan Gula Darah Biasa:** Biasanya mencakup pemeriksaan gula darah puasa dan gula darah sewaktu. Paket ini cocok untuk pengecekan rutin atau skrining awal.\n2.  **Paket Pemeriksaan Profil Gula Darah:** Lebih komprehensif, meliputi gula darah puasa, gula darah 2 jam setelah makan, dan HbA1c (untuk melihat rata-rata kadar gula darah selama 2-3 bulan terakhir). Paket ini direkomendasikan untuk diagnosis dan pemantauan diabetes.\n3.  **Paket Skrining Diabetes:** Paket ini biasanya ditawarkan untuk individu dengan faktor risiko diabetes, seperti obesitas, riwayat keluarga dengan diabetes, atau usia di atas 45 tahun.\n\nUntuk informasi lebih detail mengenai harga, persiapan sebelum pemeriksaan, dan ketersediaan paket, saya sarankan Anda untu

# Workflow for agent to use tool

In [13]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
# Define state for application
class State(TypedDict):
    question: str
    context: List[str]
    search: str
    answer: str

In [14]:
def get_context(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an expert in Medical Check-Up (MCU) packages.
                You will provide keywords about the MCU packages based on the question.
                The keywords should be relevant to the MCU packages available at Siloam hospitals.
                Do not provide any other information.
                If the question already contains keywords, you can return them as is.
                Only return one keyword and in english.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"]})
    return {"search": result.content}

In [15]:
def retrieve(state: State):
    retrieved_docs = search_mcu_packages(state["search"])
    return {"context": retrieved_docs}

In [ ]:
def generate(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an assistant that provides information about Medical Check-Up (MCU) packages at Siloam hospitals.
                You will generate a response based on the context provided.
                The response should be concise and relevant to the question asked.
                package list knowledge: 
                {context}
                If the context is empty, you can provide a general response about MCU packages.
                Please always include related packages in your response.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"], "context": state["context"]})
    return {"answer": result.content}

In [17]:
graph_builder = StateGraph(State).add_sequence([get_context, retrieve, generate])
graph_builder.add_edge(START, "get_context")
graph = graph_builder.compile()

In [21]:
response = graph.invoke({
	"question": "Saya mau cek kanker di siloam, ada paket apa aja ya ?",
	"context": [],
	"search": "",
	"answer": ""
})
print(response["answer"])

Tentu, di Siloam Hospitals kami memiliki beberapa paket Medical Check-Up (MCU) untuk deteksi kanker, diantaranya:

*   **Gastric Cancer Comprehensive:** Kombinasi gastroskopi dengan pemeriksaan tumor marker CA 72-4.
*   **Gastric & Colorectal Cancer Comprehensive:** Kombinasi endoskopi dengan pemeriksaan tumor marker seperti CEA dan CA 72-4.
*   **Pancreatic Cancer Comprehensive:** Kombinasi MRCP dengan pemeriksaan tumor marker seperti CA 19-9.
*   **Whole Body Cancer (Female) Comprehensive:** Kombinasi berbagai tes pencitraan dengan tes tumor marker untuk deteksi kemungkinan kanker pada wanita.
*   **Whole Body Cancer (Male) Comprehensive:** Kombinasi berbagai tes pencitraan dengan tes tumor marker untuk deteksi kemungkinan kanker pada pria.
*   **Colorectal Cancer Comprehensive:** Kombinasi kolonoskopi dengan pemeriksaan tumor marker CEA.
*   **Liver Cancer Comprehensive:** Kombinasi USG Abdomen dengan pemeriksaan tumor marker seperti AFP dan PIVKA II.
*   **Breast Cancer Comprehensi